So lets say we start with 10 years daily stock price data for, say,  S&P 500 stocks. The data will typically come normalised, columns being: date, stock-ticker, high, low, open, close, volume. 

Now over 10 years, different tickers will enter and leave the index. Moreover, even when a ticker is currently in the list, it may not trade on a particular day (they may be suspended for a day... this is not very common in US stocks, but fairly common in emerging market stocks). So the first order of business is fixing that. 

So say I want do run an analysis on the last 5 years data. I need a list of stocks that have been present in the index for the entire period. If I am lucky, someone has a list of starting and ending dates for index constituents. I cross reference that to my price data, to get a list of say 450 stocks that have data for the entire period (modulo trading suspensions)

Sometimes I won't have a list of index constituents. Then I will have to infer that from the data... which stocks have data for the entire period. And then do a filter for those. 

If we are doing global stocks, then there can be some other fun. Most markets trade mon-fri, while middle eastern stocks can trade sun-thursday. In this case, we might want to align the working days by week. 

Now we come to the single pieces of missing data for any particular day. Sometimes, we might want to interpolate the data. Say we have closing price, but no opening.. we can interpolate, and carry on. 

Sometimes however, if we have data missing for one stock, we might want to get rid of all stocks for that day. Alternatively, we might want to get rid of all stocks that do not have data for all trading days. Note that for doing this analysis, you cannot go by mon-fri, there are holidays, for example. 

With the data cleaned up, we can now process it. First would be create a series of daily returns : (today's closing price - yesterday's closing price)/(yesterday's price). Note that this will lose the first row of data. We might also want to create weekly returns. Sometimes, we will create returns from open/close, rather than close close... in which case, we will use two fields to compute a third. At this stage, we might also create a series of moving averages for the daily returns (this will remove the first N rows of data, for an N day moving average), and compute volatility of returns. For each stock, we may want to plot the open/high/low/close prices, the moving average, and the volume. 

Now, say we select the top 10 stocks based on volatility of returns.  We take the daily returns for these stocks as a 2D matrix, and calculate a covariance matrix for them. This covariance matrix then goes into the old JuMP notebook for portfolio optimisation

Now, say you have tick data, which is every trade price during the day. The columns will be: ticker, time, price. To do a similar analysis as above, the first thing you would need to do is bucket the prices, say every 10 minutes, or every hour. Within that period, you would calculate open/high/low/close. Note that you might have more missing data, since some stocks wouldn't trade in that period. Having done that, repeat a similar set of steps as for daily data. 

Note that there is a lot more you can do with tick data, but this is one particular plausable workflow. 

In [1]:
using DataFrames
using JuliaTS
using Base.Dates
using Vega

In [2]:
function gen_days(nyears)
    alldays = [Date(now()-Year(nyears)):Date(now());]
    daynums = [dayofweek(d) for d in alldays]
    weekdays = alldays[daynums .< 6]
    ndays = length(weekdays)

    # delete random holidays (15 per year)
    holidays = randperm(ndays)[1:(15 * nyears)]
    daymask = trues(ndays)
    daymask[holidays] = false
    weekdays[daymask]
end

function gen_volatility(dayavgs, volatility)
    ndays = size(dayavgs, 2)
    out = copy(dayavgs)
    for d in 1:ndays
        out[:,d] += out[:,d] .* volatility .* randn(500)
    end
    abs(out)
end

function gen_prices(nstocks, ndays)
    t1prices = repmat([50.:4.3:650.;], 4)[randperm(nstocks)]
    t2prices = repmat([-50.:8.7:800.;], 6)[randperm(nstocks)]
    incrprices = (t2prices .- t1prices) / ndays

    dayavgs = Array(Float64, nstocks, ndays)
    dayavgs[:,1] = t1prices
    for d in 2:ndays
        prevday = dayavgs[:,d-1]
        dayavgs[:,d] = prevday .+ incrprices
    end
    dayavgs
end

function generate_data()
    scriptids = [1:500;]

    weekdays = gen_days(10)
    ndays = length(weekdays)
    dayavgs = gen_prices(500, ndays)
    volatility = repmat([0.0:0.5:20.0;], 30)[1:500] / 100

    opens = gen_volatility(dayavgs, volatility)
    closes = gen_volatility(dayavgs, volatility)
    highs = gen_volatility(dayavgs, volatility)
    lows = gen_volatility(dayavgs, volatility)
    highs = max(opens, closes, highs, lows)
    lows = min(opens, closes, highs, lows)
    
    col_date = sort(repmat(weekdays, 500))
    col_script = repmat(scriptids, ndays)
    col_opens = reshape(opens, length(opens))
    col_closes = reshape(closes, length(opens))
    col_highs = reshape(highs, length(opens))
    col_lows = reshape(lows, length(opens))
    
    TArray((:script,:date), :date=>col_date, :script=>col_script, :open=>col_opens,
    :close=>col_closes, :high=>col_highs, :low=>col_lows)
end

generate_data (generic function with 1 method)

In [16]:
data = generate_data()

Tuple{Int64,Date} => Tuple{Float64,Float64,Float64,Float64}

 (1,2006-05-08) => (187.6,187.6,187.6,187.6)
 (1,2006-05-09) => (187.69439024390243,187.69439024390243,187.69439024390243,187.69439024390243)
 (1,2006-05-10) => (187.78878048780487,187.78878048780487,187.78878048780487,187.78878048780487)
 (1,2006-05-11) => (187.8831707317073,187.8831707317073,187.8831707317073,187.8831707317073)
 (1,2006-05-12) => (187.97756097560975,187.97756097560975,187.97756097560975,187.97756097560975)
 (1,2006-05-15) => (188.0719512195122,188.0719512195122,188.0719512195122,188.0719512195122)
 (1,2006-05-16) => (188.16634146341462,188.16634146341462,188.16634146341462,188.16634146341462)
 (1,2006-05-17) => (188.26073170731706,188.26073170731706,188.26073170731706,188.26073170731706)
 (1,2006-05-18) => (188.3551219512195,188.3551219512195,188.3551219512195,188.3551219512195)
 (1,2006-05-22) => (188.44951219512194,188.44951219512194,188.44951219512194,188.44951219512194)
 ⋮
 (500,2016-04-25) => (505.6824

TArray 1230000x6  (:script,:date) => (:open,:close,:high,:low)

In [4]:
function plotprices(data::TArray, stock, pricecol)
    stockprice = data[stock, Date("2006-01-01"):Date("2016-12-31")]
    prices = getvals(stockprice, pricecol)
    lineplot(x=1:length(prices), y=prices)
end

plotprices (generic function with 1 method)

In [17]:
plotprices(data, 22, :open)

In [6]:
returnsfn(d1, d0) = (100 * (d1[1] - d0[1]) / d0[1],)

function dailyreturns(data::TArray)
    sdata = timeshift(data, :date, Day(-1))
    taout = TArray(data.keynames, :script=>Int[], :date=>Date[], :returns=>Float64[])
    d1 = project(data, data.keynames, (:open,))
    d2 = project(data, data.keynames, (:close,))
    naturaljoin(taout, d1, d2, :inner, returnsfn)
end

dailyreturns (generic function with 1 method)

In [18]:
drets = dailyreturns(data)

Tuple{Int64,Date} => Tuple{Float64}

 (1,2006-05-08) => (0.0,)
 (1,2006-05-09) => (0.0,)
 (1,2006-05-10) => (0.0,)
 (1,2006-05-11) => (0.0,)
 (1,2006-05-12) => (0.0,)
 (1,2006-05-15) => (0.0,)
 (1,2006-05-16) => (0.0,)
 (1,2006-05-17) => (0.0,)
 (1,2006-05-18) => (0.0,)
 (1,2006-05-22) => (0.0,)
 ⋮
 (500,2016-04-25) => (-9.288947330306234,)
 (500,2016-04-26) => (-11.322836697041984,)
 (500,2016-04-27) => (3.556894659968372,)
 (500,2016-04-28) => (-0.854425088158863,)
 (500,2016-04-29) => (-3.8588011233389143,)
 (500,2016-05-02) => (4.105564610903701,)
 (500,2016-05-03) => (4.327436595120575,)
 (500,2016-05-04) => (13.221775905004941,)
 (500,2016-05-05) => (-6.702274802977447,)
 (500,2016-05-06) => (0.5388626998190026,)

TArray 1230000x3  (:script,:date) => (:returns,)

In [19]:
plotprices(drets, 22, :returns)

In [9]:
# create moving average of daily returns
function movingavg(data::TArray, ndays)
    taout = TArray(data.keynames, :script=>Int[], :date=>Date[], :returns=>Float64[])
    d1, d2 = extrema(getvals(data, :date))
    wspec = Window((1,d1), (500,d2), (1,Day(1)), (0, Day(ndays)))
    window(taout, data, wspec, :first, mean)
    taout
end

movingavg (generic function with 1 method)

In [20]:
movavg = movingavg(drets, 5)

Tuple{Int64,Date} => Tuple{Float64}

 (1,2006-05-08) => (0.0,)
 (1,2006-05-09) => (0.0,)
 (1,2006-05-10) => (0.0,)
 (1,2006-05-11) => (0.0,)
 (1,2006-05-12) => (0.0,)
 (1,2006-05-13) => (0.0,)
 (1,2006-05-14) => (0.0,)
 (1,2006-05-15) => (0.0,)
 (1,2006-05-16) => (0.0,)
 (1,2006-05-17) => (0.0,)
 ⋮
 (500,2016-04-26) => (-3.119792062142847,)
 (500,2016-04-27) => (0.737308264843574,)
 (500,2016-04-28) => (0.9299437486316247,)
 (500,2016-04-29) => (4.448993996922576,)
 (500,2016-04-30) => (3.738125577012943,)
 (500,2016-05-01) => (3.098273001574155,)
 (500,2016-05-02) => (3.098273001574155,)
 (500,2016-05-03) => (2.846450099241768,)
 (500,2016-05-04) => (2.3527879339488322,)
 (500,2016-05-05) => (-3.0817060515792223,)

TArray 1825999x3  (:script,:date) => (:returns,)

In [21]:
plotprices(movavg, 22, :returns)

In [12]:
# compute volatility
function volatility{T<:DatePeriod}(data::TArray, interval::T)
    taout = TArray(data.keynames, :script=>Int[], :date=>Date[], :volatility=>Float64[])
    d1, d2 = extrema(getvals(data, :date))
    wspec = Window((1,d1), (500,d2), (1,zero(T)), (0,interval))
    window(taout, data, wspec, :first, std)
    taout
end

volatility (generic function with 1 method)

In [22]:
volat = volatility(movavg, Year(20))

Tuple{Int64,Date} => Tuple{Float64}

 (1,2006-05-08) => (0.0,)
 (2,2006-05-08) => (0.3540695974893476,)
 (3,2006-05-08) => (0.7265071425840356,)
 (4,2006-05-08) => (1.096631345882692,)
 (5,2006-05-08) => (1.456243714353436,)
 (6,2006-05-08) => (1.8087152362395975,)
 (7,2006-05-08) => (2.128130683828496,)
 (8,2006-05-08) => (2.5318498162572625,)
 (9,2006-05-08) => (2.6953026859118583,)
 (10,2006-05-08) => (3.087302066953429,)
 ⋮
 (491,2006-05-08) => (15.592463551249363,)
 (492,2006-05-08) => (15.832986439067353,)
 (493,2006-05-08) => (0.0,)
 (494,2006-05-08) => (0.3427444164291981,)
 (495,2006-05-08) => (0.7190672042212111,)
 (496,2006-05-08) => (1.0454533502927472,)
 (497,2006-05-08) => (1.4526404496224066,)
 (498,2006-05-08) => (1.8153795746663435,)
 (499,2006-05-08) => (2.2467688313335814,)
 (500,2006-05-08) => (2.438543421558828,)

TArray 500x3  (:script,:date) => (:volatility,)

In [23]:
x=getvals(volat, :script)
y=getvals(volat, :volatility)
print(mean(y), ", ", maximum(y), ", ", median(y))

7.526749194443787, 17.27541036967488, 7.140682571790615

In [24]:
mask = 14 .< y .< 18
x = x[mask]
y = y[mask] .- 14
barplot(x=x, y=y)